In [1]:
import os
os.environ["OPENAI_API_KEY"] = "voc-8162499801266773377505669655d3c05508.40840521"
os.environ["OPENAI_API_BASE"] = "https://openai.vocareum.com/v1"

In [12]:
import json
import re
from pathlib import Path
import jq
from langchain_community.document_loaders import JSONLoader
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma

In [36]:
file_path='./generated_adverts_a.jsonl'

In [ ]:
with open(file_path, "r") as file:
    for line in file:
        data_entry = json.loads(line)
        # Process each data_entry as a Python dict
        print(data_entry)

In [38]:
loader = JSONLoader(
    file_path=file_path,
    jq_schema='.',
    text_content=False,
    json_lines=True)
docs = loader.load()

In [ ]:
print(docs[0])

In [ ]:
def json_advert_to_string(doc):
    json_to_str = json.dumps(json.loads(doc.page_content), indent=0)
    json_to_str = json_to_str.replace("\"", "").replace(",\n", "\n")
    json_to_str = json_to_str.replace("{\n", "").replace("\n}", "")
    json_to_str = re.sub(r"Index: \d+\n", "", json_to_str)
    return json_to_str


print(json_advert_to_string(docs[0]))

In [ ]:
embeddings = OpenAIEmbeddings()
vector_store = Chroma(
    collection_name="adverts",
    embedding_function=embeddings,
    persist_directory="./chroma_langchain_db",  # Where to save data locally, remove if not neccesary
)